### Import Packages

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sqlean as sqlite3
import spacy

In [4]:
nlp = spacy.load('en_core_web_sm')

### Import Data

In [5]:
DATABASE_PATH = 'C:/University/6G7V0007_MSC_Project/Project/Data/joblistings_transformed.db'
con = sqlite3.connect(DATABASE_PATH)

In [6]:
job = pd.read_sql('SELECT * FROM job', con)
company = pd.read_sql('SELECT * FROM company', con)
website = pd.read_sql('SELECT * FROM website', con)

C:\Users\walte\AppData\Local\Temp\ipykernel_3640\1361633401.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  job = pd.read_sql('SELECT * FROM job', con)
C:\Users\walte\AppData\Local\Temp\ipykernel_3640\1361633401.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  company = pd.read_sql('SELECT * FROM company', con)
C:\Users\walte\AppData\Local\Temp\ipykernel_3640\1361633401.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  website = pd.read_sql('SELECT * FROM website', con)


In [7]:
job.set_index('id', inplace=True)
company.set_index('id', inplace=True)
website.set_index('id', inplace=True)

In [8]:
test = job.merge(company, left_on='company_id', right_index=True)

In [9]:
test.rename(columns={'name':'company_name'}, inplace=True)

In [10]:
test.head()

,website_id,company_id,title,location,pay,description,timestamp,company_name
id,,,,,,,,
1,1,1,Graduate Electronics/Computer Science Software...,Cirencester (Gloucestershire),"£30,000",About the role A unique opportunity to join a ...,2024-06-28,AccuLink Technology
145,1,1,Project Engineer,Cirencester (Gloucestershire),Competitive,AccuLink Technology specialises in the design ...,2024-06-28,AccuLink Technology
2,1,2,Graduate Software Development Engineer,London,Competitive,nFocus Graduate Academy Accelerate your career...,2024-06-28,nfocus
3,1,3,Graduate Technology Consultant,Manchester (Hybrid),Competitive,About Arlanis Reply: Arlanis Reply is the Repl...,2024-06-28,Reply
20,1,3,Graduate Software Developer,London and Manchester (Hybrid),Competitive,About Arlanis Reply: Arlanis Reply is the Repl...,2024-06-28,Reply


### Preprocessing for TF-IDF

In [ ]:
# Set tokeniser to use only whitespace to split
nlp.tokenizer = 

In [ ]:
def tokenise(text):
    """Tokenise text"""
    return [token.text for token in nlp(text)]

In [22]:
def lemmatise(text):
    """Tokenise and lemmatise text"""
    return [token.lemma_ for token in nlp(text)]

In [25]:
def get_entities(text):
    """Return the named entities"""
    return [(ent.text, ent.label_) for ent in nlp(text).ents]

In [26]:
test['ents'] = test['description'].apply(get_entities)

In [23]:
# test['doc'] = test['description'].apply(preprocess)

In [35]:
test['ents'].iloc[6]

[('the Reply Group', 'ORG'),
 ('SAP', 'ORG'),
 ('Customer', 'ORG'),
 ('Portaltech Reply', 'ORG'),
 ('eCommerce', 'PRODUCT'),
 ('eCommerce', 'PRODUCT'),
 ('2000', 'DATE'),
 ('Portaltech Reply', 'ORG'),
 ('eCommerce', 'PRODUCT'),
 ('hybris', 'PRODUCT'),
 ('2007', 'DATE'),
 ('hybris Global Partner', 'PERSON'),
 ('the Year 2012', 'EVENT'),
 ('2011', 'DATE'),
 ('2009', 'DATE'),
 ('SAP Hybris', 'ORG'),
 ('2015', 'DATE'),
 ('Portaltech', 'GPE'),
 ('SAP', 'ORG'),
 ('the Travel Accelerator', 'ORG'),
 ('We Reply Sometimes', 'ORG'),
 ('Develop Java', 'PERSON'),
 ('Selenium', 'ORG'),
 ('Cucumber BDD', 'ORG'),
 ('Verify', 'PRODUCT'),
 ('Raise', 'PERSON'),
 ('Qualifications A', 'ORG'),
 ('2.1', 'CARDINAL'),
 ('Bachelor', 'ORG'),
 ('1+ years', 'DATE'),
 ('Java', 'PERSON'),
 ('1+ years', 'DATE'),
 ('Automation', 'GPE'),
 ('Git', 'GPE'),
 ('SAP Commerce', 'ORG'),
 ('Hybris', 'PRODUCT'),
 ('API Automation', 'ORG'),
 ('CI', 'PERSON'),
 ('Jenkins', 'PERSON'),
 ('Benefits Extensive', 'WORK_OF_ART'),
 ('Rep

In [24]:
test['doc'].iloc[0]

['about',
 'the',
 'role',
 'a',
 'unique',
 'opportunity',
 'to',
 'join',
 'a',
 'grow',
 'new',
 'company',
 'develop',
 'some',
 'exciting',
 'product',
 'in',
 'the',
 '5',
 'g',
 'mobile',
 'infrastructure',
 'space',
 '.',
 'as',
 'a',
 'recent',
 'graduate',
 'you',
 'would',
 'be',
 'join',
 'an',
 'r&d',
 'team',
 'involve',
 'in',
 'the',
 'design',
 ',',
 'development',
 'and',
 'manufacture',
 'of',
 'these',
 'product',
 '.',
 'the',
 'role',
 'will',
 'be',
 'able',
 'to',
 'take',
 'the',
 'skill',
 'learn',
 'during',
 'your',
 'degree',
 'and',
 'apply',
 'they',
 'to',
 'real',
 'world',
 'application',
 '.',
 'work',
 'from',
 'requirement',
 'you',
 'will',
 'be',
 'expect',
 'to',
 'develop',
 'all',
 'form',
 'of',
 'software',
 'across',
 'the',
 'full',
 'suite',
 ';',
 'from',
 'firmware',
 'on',
 'microprocessor',
 'such',
 'as',
 'the',
 'Raspberry',
 'PI',
 ',',
 'with',
 'code',
 'write',
 'in',
 "'",
 'c',
 "'",
 'and/or',
 'Python',
 ',',
 'all',
 'the',